This workbook is primarily to get the data from the sqlite3 db, matches table and create the train and test the models.
There are many models implemented , uncomment the appropriate one and run

In [1]:
import numpy as np
from sklearn import linear_model
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import svm
from sklearn import neighbors
#from sklearn.neural_network import MLPClassifier
from sklearn import preprocessing
import math
import time

from sklearn.externals import joblib

import simple_heroes

import sqlite3
import os
import os.path

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
119
120


In [2]:
def extract_standard(radiant_won, radiant, dire):
    """
    Turn a match constisting of its winner and the radiant and dire heroes into a feature vector.
    
    The default version uses a 224 dimensional feature vector and sets elements corresponding to the picked heroes to 1 while the rest are set to 0.
    The first 112 elements are the heroes on the radiant team and the other the heroes on the dire team.
    
    The target class 0 represents the radiant winning, and 1 the dire winning.
    """
    def put_heroes_in_features(heroes, features):
        for i in heroes:
            features[simple_heroes.real_to_ordered(i)] = 1.0

    winner = int(not radiant_won)

    radiant_heroes = [0] * len(simple_heroes.dota_hero_ids)
    dire_heroes = [0] * len(simple_heroes.dota_hero_ids)

    put_heroes_in_features(radiant, radiant_heroes)
    put_heroes_in_features(dire, dire_heroes)

    features = radiant_heroes + dire_heroes

    return (winner, features)

def load_data(database, undersample=True):
    """
    Load data from a sqlite3 database and convert the matches into feature vectors in form of numpy arrays.
    
    Optionally perform undersampling to get the same number of samples of both classes.
    
    This method can be used directly, or indirectly via make_training_validate_test.
    """
    connection = sqlite3.connect(database)
    if undersample:
        number_of_radiant_wins = connection.execute('SELECT COUNT(radiant_win) FROM matches WHERE radiant_win IS NOT NULL AND radiant_win = 1 AND has_leaver = 0 AND radiant_hero_1 != 0 AND radiant_hero_2 != 0 AND radiant_hero_3 != 0 AND radiant_hero_4 != 0 AND radiant_hero_5 != 0 AND dire_hero_1 != 0 AND dire_hero_2 != 0 AND dire_hero_3 != 0 AND dire_hero_4 != 0 AND dire_hero_5 != 0').fetchone()[0]
        number_of_dire_wins = connection.execute('SELECT COUNT(radiant_win) FROM matches WHERE radiant_win IS NOT NULL AND radiant_win = 0 AND has_leaver = 0 AND radiant_hero_1 != 0 AND radiant_hero_2 != 0 AND radiant_hero_3 != 0 AND radiant_hero_4 != 0 AND radiant_hero_5 != 0 AND dire_hero_1 != 0 AND dire_hero_2 != 0 AND dire_hero_3 != 0 AND dire_hero_4 != 0 AND dire_hero_5 != 0').fetchone()[0]
        max_games_per_class = min(number_of_radiant_wins, number_of_dire_wins)
        radiant_wins = 0
        dire_wins = 0
    dataset = list()
    targetset = list()
    for lineno, line in enumerate(connection.execute('SELECT radiant_win, radiant_hero_1,radiant_hero_2,radiant_hero_3,radiant_hero_4,radiant_hero_5, dire_hero_1,dire_hero_2,dire_hero_3,dire_hero_4,dire_hero_5 FROM matches WHERE radiant_win IS NOT NULL AND has_leaver = 0 AND radiant_hero_1 != 0 AND radiant_hero_2 != 0 AND radiant_hero_3 != 0 AND radiant_hero_4 != 0 AND radiant_hero_5 != 0 AND dire_hero_1 != 0 AND dire_hero_2 != 0 AND dire_hero_3 != 0 AND dire_hero_4 != 0 AND dire_hero_5 != 0')):
        radiant_won = line[0]

        if undersample:
            if radiant_wins == max_games_per_class and dire_wins == max_games_per_class: #both classes already at max games
                break
            elif radiant_won == 0: #dire won
                if dire_wins == max_games_per_class: #dire already at max games
                    continue
                else:
                    dire_wins += 1
            else: #radiant won
                if radiant_wins == max_games_per_class: #radiant already at max games
                    continue
                else:
                    radiant_wins += 1

        radiant_hero_ids = line[1:6]
        dire_hero_ids  = line[6:]
        winner, features = extract_standard(radiant_won, radiant_hero_ids, dire_hero_ids)
        targetset.append(winner)
        dataset.append(features)
    connection.close()
    
    samples = np.array(dataset)
    target = np.array(targetset)
    # http://stackoverflow.com/questions/4601373/better-way-to-shuffle-two-numpy-arrays-in-unison
    rng_state = np.random.get_state()
    np.random.shuffle(samples)
    np.random.set_state(rng_state)
    np.random.shuffle(target)
    return (samples, target)


def make_training_validate_test(database, training_ratio, undersample=True):
    """
    Create a training, validate and test set to be used in model parameter tuning.
    
    Specified is the amount of samples in the training set in form of a ratio. The validate and test sets use remaining samples split evenly between them.
    
    The results are written to disk in the data directory.
    """
    assert(training_ratio > 0 and training_ratio < 1)
    validate_ratio = test_ratio = (1-training_ratio) / 2
    (data, target) = load_data(database, undersample=undersample)
    
    number_of_samples = len(target)
    number_of_training_samples = math.floor(number_of_samples * training_ratio)
    number_of_validate_samples = math.floor(number_of_samples * validate_ratio)
    number_of_test_samples = math.floor(number_of_samples * test_ratio)
    print(number_of_training_samples)
    print('number_of_validate_samples:')
    print(number_of_validate_samples)
    training_data = data[0:number_of_training_samples]
    training_target = target[0:number_of_training_samples]
    validate_data = data[number_of_training_samples:number_of_training_samples+number_of_validate_samples]
    validate_target = target[number_of_training_samples:number_of_training_samples+number_of_validate_samples]
    test_data = data[number_of_training_samples+number_of_validate_samples:number_of_training_samples+number_of_validate_samples+number_of_test_samples]
    test_target = target[number_of_training_samples+number_of_validate_samples:number_of_training_samples+number_of_validate_samples+number_of_validate_samples]
    
    joblib.dump(training_data, "data/training_data");
    joblib.dump(training_target, "data/training_target");
    joblib.dump(validate_data, "data/validate_data");
    joblib.dump(validate_target, "data/validate_target");
    joblib.dump(test_data, "data/test_data");
    joblib.dump(test_target, "data/test_target");


def export_model(model, name):
    """Export a model to disk. Models can consist of multiple files so a directory is created for each model."""
    path = "data/{}/".format(name)
    filename = "{}.model".format(name)
    if os.path.isdir(path):
        print("model already exists")
        return
    else:
        os.mkdir(path)
        joblib.dump(model, path + filename)
        
def load_model(name):
    """Load a model from disk."""
    model = joblib.load("data/{}/{}.model".format(name, name))
    # Setting n_jobs to 1 in case it was set to a higher number while training the model seems to makes predictions of single samples much faster.
    model.n_jobs = 1
    return model

def print_speed_and_accuracy(models = ["decisiontree", "gradienttreeboosting", "knn", "logisticregression", "randomforest"]):
    """
    Helper method to print speed and accuracy of already created models on the test data.
    
    Speed is measured in the time it takes to predict 1000 samples sequentially.
    """
    test_data = joblib.load("data/test_data")
    test_target = joblib.load("data/test_target")

    for model in models:
        model = load_model(model)
        accuracy = model.score(test_data, test_target)
        start = time.time()
        for i in range(1000):
            model.predict_proba([test_data[i]])
        end = time.time()
        duration = end - start
        print(model, accuracy, duration)




In [3]:
make_training_validate_test("dota2data", 0.8)

10870.0
number_of_validate_samples:
1358.0


C:\Users\Madhukar\Anaconda2\lib\site-packages\ipykernel\__main__.py:96: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
C:\Users\Madhukar\Anaconda2\lib\site-packages\ipykernel\__main__.py:97: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
C:\Users\Madhukar\Anaconda2\lib\site-packages\ipykernel\__main__.py:98: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
C:\Users\Madhukar\Anaconda2\lib\site-packages\ipykernel\__main__.py:99: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
C:\Users\Madhukar\Anaconda2\lib\site-packages\ipykernel\__main__.py:100: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
C:\Users\Madhukar\Anaconda2\lib\site-packages\ipykernel\__main__.py:1

In [4]:


training_data = joblib.load("data/training_data")
training_target = joblib.load("data/training_target")

validate_data = joblib.load("data/validate_data")
validate_target = joblib.load("data/validate_target")

test_data = joblib.load("data/test_data")
test_target = joblib.load("data/test_target")


In [5]:


#training a model
model = linear_model.LogisticRegression(solver='sag', dual=False, penalty='l2', n_jobs=4)
modelname = "logisticregression"

model.fit(training_data, training_target)
model.score(validate_data, validate_target)


0.63549337260677463

In [14]:


#storing the model
export_model(model, modelname)

In [6]:
#training a model
model = RandomForestClassifier(n_estimators=200, min_samples_split=0.001, max_depth=None, max_features="auto", n_jobs=2)
modelname = "random_forest_classifier"

model.fit(training_data, training_target)
model.score(validate_data, validate_target)


0.62002945508100149

In [16]:
export_model(model, modelname)

In [7]:
#training a model
model = GradientBoostingClassifier(n_estimators=75, max_features=None, max_depth=15)
modelname = "gradient_Boosting_classifier"

model.fit(training_data, training_target)
model.score(validate_data, validate_target)


0.57363770250368185

In [18]:
export_model(model, modelname)

In [19]:
#training a model
model = neighbors.KNeighborsClassifier(algorithm='ball_tree', n_neighbors=180, weights='distance', metric='manhattan', leaf_size=200, n_jobs=4)
modelname = "K_Neighbhors_classifier"

model.fit(training_data, training_target)
model.score(validate_data, validate_target)

0.60382916053019142

In [20]:
export_model(model, modelname)

In [21]:
from sklearn.neural_network import MLPClassifier
model = MLPClassifier(hidden_layer_sizes=(100,))
modelname = "MLPCLassifier"

model.fit(training_data, training_target)
model.score(validate_data, validate_target)

0.57731958762886593

In [22]:
export_model(model, modelname)